In [1]:
import mysql.connector
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import os
%matplotlib inline 

In [2]:
user_name=os.environ.get('DB_USER')
passwd=os.environ.get('DB_PASS')
print(user_name)
print(passwd)

None
None


In [10]:
connection = mysql.connector.connect(host ='cpanel.insaid.co',user = "student",passwd="student",db='Capstone1')

In [11]:
connection

In [12]:
emp_table=pd.read_sql_query('select * from phone_brand_device_model',connection)
emp_table

,device_id,phone_brand,device_model
0,1877775838486905855,vivo,Y13
1,-3766087376657242966,小米,V183
2,-6238937574958215831,OPPO,R7s
3,8973197758510677470,三星,A368t
4,-2015528097870762664,小米,红米Note2
...,...,...,...
87721,-4961458925928574032,华为,荣耀畅玩4X
87722,-8819817317449262375,华为,荣耀6
87723,-3358291377416935026,华为,荣耀畅玩4
87724,3282788959750981633,小米,MI 2


In [13]:
dept_name=pd.read_sql_query('select * from gender_age_train',connection)
dept_name 

,device_id,gender,age,group
0,-8076087639492063270,M,35,M32-38
1,-2897161552818060146,M,35,M32-38
2,-8260683887967679142,M,35,M32-38
3,-4938849341048082022,M,30,M29-31
4,245133531816851882,M,30,M29-31
...,...,...,...,...
74640,4682031842235089751,M,30,M29-31
74641,-9178703742877135986,M,30,M29-31
74642,180946546684162312,M,20,M22-
74643,1390702386071991851,M,37,M32-38


In [ ]:
def cal_tax(sal):
        tax=0
        if sal>500 and sal<=1250:
            tax=sal * .125
        elif sal>1250 and sal<=1700:
            tax=sal*.175
        elif sal>1700 and sal<=2500:
            tax=sal*.225
        elif sal >2500:
            tax= sal *.275
        else:
             tax=0
        return tax 
    
        

In [ ]:
emp_table['SAL']

In [ ]:
emp_table['Tax']=emp_table['SAL'].map(cal_tax)
emp_table

In [ ]:
def Commission(comm):
        commision=0
        if comm == "NaN":
         commision = emp_table['SAL'] * 1
        else:
         commision= comm *1
        return commision
         

In [ ]:
emp_table['new_comm']=emp_table['COMM'].map(Commission)
emp_table

In [ ]:
emp_table.drop(columns='new_comm', axis=1)

In [ ]:
emp_table['comm']=emp_table[['SAL','COMM']].apply(lambda x: x[0]if pd.isnull(x[1]) or int(x[1]==0) else x[1],axis=1)

In [ ]:
emp_table.drop(columns='new_comm',axis=1,inplace=True)

In [ ]:
emp_table

In [ ]:
##finding comm/sal percentage
emp_table['comm_%']=emp_table['comm']/emp_table['SAL'] * 100

In [ ]:
emp_table

In [ ]:
emp_table['Comm_flag']=emp_table[['SAL','comm']].apply(lambda x: 'invalid' if int(x[0])==int(x[1]) else 'valid',axis=1)

In [ ]:
emp_table.head(25)

In [ ]:
comm_valid=emp_table[emp_table['Comm_flag']=='valid']
comm_valid

In [ ]:
comm_invalid=emp_table[emp_table['Comm_flag']=='invalid']
comm_invalid

In [ ]:
agg_sal=emp_table.groupby(['DEPTNO'])['SAL'].mean()
agg_sal

In [ ]:
joinedframe=pd.merge(emp_table,agg_sal,on="DEPTNO",how='inner')
joinedframe

In [ ]:
joinedframe

In [ ]:
joinedframe.rename(columns={'SAL_x':'sal','SAL_y':'avg_sal'},inplace=True)
#joinedframe.drop(columns=['Comm_flag'],inplace=True,axis=1)
joinedframe.sort_values('DEPTNO')

In [ ]:
job_map={'MANAGER':'MGR','PRESIDENT':'Country_Head','CLERK':'CLK',
 'ANALYST':'SDE2','SALESMAN':'Sales&Marketing'}
df=joinedframe.replace({'JOB':job_map})
df

In [ ]:
final=pd.merge(df,dept_name[['DEPTNO','DNAME','LOC']],on='DEPTNO',how='inner')
final

In [ ]:
dname_map={'RESEARCH':'R&D','SALES':'SALES','ACCOUNTING':'ACCT'}
dff=final.replace({'DNAME':dname_map})
dff

In [ ]:
cleaned_df=dff[['EMPNO','ENAME','JOB','HIREDATE','sal','Tax','avg_sal','DNAME','LOC']]
cleaned_df